In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Instalasi dependensi
%%capture
!pip install unsloth nltk datasets trl transformers

# Import library
from unsloth import FastLanguageModel, is_bfloat16_supported # Import is_bfloat16_supported here
import torch
import pandas as pd
from nltk.tokenize import sent_tokenize
from datasets import Dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer

# Unduh resource 'punkt' untuk NLTK
import nltk
nltk.download('punkt')

In [5]:
# Konfigurasi dasar
max_seq_length = 2000

# Load model dan tokenizer default
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=False,
)

# Konfigurasi LoRA untuk fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.2,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
# Load dataset dari CSV
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset - Sheet1 (2).csv")
df = pd.DataFrame(data)

# Drop kolom yang tidak diperlukan
df.drop(columns=["input"], inplace=True)

nltk.download('punkt_tab')
# Terapkan sentence tokenization pada kolom 'instruction' dan 'output'
def split_into_sentences(text):
    return sent_tokenize(text)

df['instruction'] = df['instruction'].apply(split_into_sentences)
df['output'] = df['output'].apply(split_into_sentences)

# Format ulang kolom 'conversations' dengan sentence tokenization
df['conversations'] = df.apply(lambda row: [
    {"from": "system", "value": "Sistem ini dibuat oleh salsa data terbaru 2025"},
    {"from": "human", "value": " ".join(row['instruction'])},  # Gabungkan kalimat-kalimat
    {"from": "gpt", "value": " ".join(row['output'])}          # Gabungkan kalimat-kalimat
], axis=1)

# Hapus kolom yang tidak diperlukan lagi
df.drop(columns=["instruction", "output"], inplace=True)

# Konversi DataFrame ke format Dataset Hugging Face
dataset = Dataset.from_pandas(df)

In [7]:
# Fungsi untuk memformat prompts dengan sentence tokenization
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = []
    for convo in convos:
        formatted_convo = []
        for message in convo:
            # Terapkan sentence tokenization pada setiap pesan
            sentences = sent_tokenize(message["value"])
            formatted_convo.extend(sentences)  # Tambahkan setiap kalimat
        texts.append(" ".join(formatted_convo))  # Gabungkan kalimat-kalimat
    return {"text": texts}

# Terapkan fungsi formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

In [8]:
# Konfigurasi trainer untuk fine-tuning
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=5,
        max_steps=-1,
        learning_rate=2e-4,
        fp16=True,  # Explicitly enable fp16
        bf16=False,  # Explicitly disable bf16
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/359 [00:00<?, ? examples/s]

In [9]:
# Mulai fine-tuning
trainer_stats = trainer.train()

Step,Training Loss
1,3.276200
2,3.095100
3,3.265800
4,3.101300
5,2.777400
6,2.340400
7,2.248100
8,1.922900
9,1.907900
10,1.448600


In [ ]:
# Evaluasi model setelah fine-tuning
messages = [
    {"role": "user", "content": "Berapa poin yang didapatkan setiap anggota tim yang mendapatkan juara II di GEMASTIK?"},
]

# Tokenisasi input dengan sentence tokenization
input_sentences = sent_tokenize(messages[0]["content"])
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": " ".join(input_sentences)}],
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate output dari model
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.0,  
    top_p=0.9,        
)

# Decode output
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['system\n\nCutting Knowledge Date: December 2023\nToday Date: 23 Apr 2025\n\nuser\n\nBerapa poin yang didapatkan setiap anggota tim yang mendapatkan juara II di GEMASTIK?assistant\n\nSetiap anggota tim yang mendapatkan juara II di GEMASTIK mendapatkan 85 poin sebagai bentuk penghargaan atas pencapaiannya.']


In [27]:
# Contoh: Cek tokenized input
input_text = "Berapa poin yang didapatkan ketua tim yang mendapatkan juara I di GEMASTIK?"
tokenized_input = tokenizer(input_text, return_tensors="pt")
print(tokenized_input)

{'input_ids': tensor([[128000,  39379,  18826,    281,   2003,  10587,   1550,  91643,  32381,
           4381,   6935,  10587,  45876,  91643,  10479,   5169,    358,   1891,
            480,   2783,   6483,  29661,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [28]:
# Contoh: Uji model dengan data dari dataset training
test_input = dataset[0]['text']  # Ambil satu contoh dari dataset
inputs = tokenizer(test_input, return_tensors="pt").to("cuda")
outputs = model.generate(inputs['input_ids'], max_new_tokens=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Sistem ini dibuat oleh salsa data terbaru 2025 Berapa poin yang didapatkan ketua tim yang lolos tahap pertama (seleksi universitas) di GEMASTIK? Ketua tim yang lolos tahap pertama (seleksi universitas) di GEMASTIK mendapatkan 15 poin sebagai bentuk penghargaan atas keberhasilannya melanjutkan ke tahap berikutnya. Poin ini diberikan sebagai bentuk penghargaan atas keberhasilan tim yang lolos tahap pertama (seleksi universitas) di GEMASTIK. Poin ini diberikan sebagai bentuk penghargaan atas keberhasilan tim yang lolos tahap pertama (


In [30]:
messages = [
    {"role": "system", "content": "Sistem ini dibuat oleh salsa data terbaru 2025"},
    {"role": "user", "content": "Berapa poin yang didapatkan anggota tim yang mendapatkan juara III di ajang seni tingkat internasional?"},
]

formatted_input = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(formatted_input)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 23 Apr 2025

Sistem ini dibuat oleh salsa data terbaru 2025<|eot_id|><|start_header_id|>user<|end_header_id|>

Berapa poin yang didapatkan anggota tim yang mendapatkan juara III di ajang seni tingkat internasional?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


